# Sequence to sequence modeling with nn.transformer and Torchtext

References: https://pytorch.org/tutorials/beginner/transformer_tutorial.html

Run in [Colab](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/dca13261bbb4e9809d1a3aa521d22dd7/transformer_tutorial.ipynb?hl=en)

This a tutorial on hơ to train a sequence-to-sequence model that ues the `nn.Transformer` module.

PyTorch 1.2 release includes a standard transformer module based on the paper [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf). The transformer model has been proved to be superior in quality for many sequence-to-sequence problems while being more parallelizable. The `nn.Transformer` module relies entirely on an attention mechanism (another module recently implemented as [nn.MultiheadAttention](https://pytorch.org/docs/master/nn.html?highlight=multiheadattention#torch.nn.MultiheadAttention) to draw global dependencies between input and output. The `nn.Transformer` module is now highly modularized such that a single component (like [nn.TransformerEncoder](https://pytorch.org/docs/master/nn.html?highlight=nn%20transformerencoder#torch.nn.TransformerEncoder) in this tutorial) can be easily `adapted/composed`.

## Define the model

In this tutorial, we train `nn.TransformerEncoder` model on a language modeling ttask. The language modeling task to assign a probability for the likeehood of given word (or a sequence of words) to follow a sequence of words. A sequence of tokens are passed to the embedding layer first, followed by a positional encoding layer to account for the order of the world (see the next paragraph for more details). The `nn.TransformerEncoder` consists of multiple layers of [`nn.TransformerEncoderLayer`](https://pytorch.org/docs/master/nn.html?highlight=transformerencoderlayer#torch.nn.TransformerEncoderLayer). Along with the output sequence, a square attention mask is required because the self-attention layers in `nn.TransformerEncoder` are only allowed to attend the earlier positions in the sequence. For the language modeling task, any tokens on the future positions should be masked. To have the actual words, the output of` nn.TransformerEncoder` model is sent to the final Linear layer, which is followed by a log-Softmax function.

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropouto)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhidm, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)
        
        self.init_weights()
        
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz,sz)) == 1).transpose(0,1)
        mask = mask.float().masked_fill(mask==0, float('-inf')).masked_fill(mask==1, float(0.0))
    
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data_zero()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != src.size(0):
            device = src.device
            mask = self._generate_square_subsequent_mask(src.size(0)).to(device)
            self.src_mask = mask
        src = self.encoder(src)*math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = sefl.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output